In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import nltk
%matplotlib inline

from bs4 import BeautifulSoup
import requests
import re
import pyinputplus as pyip
import requests
import datetime as dt
from datetime import datetime
from tqdm import tqdm
import pickle
import random
import math 
import string

%load_ext autoreload
%autoreload 2

tqdm.pandas()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
# Import other files
%run climbconstants.py
%run unique_route_handling.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


--- IMPORT ---

In [57]:
upload_link = 'https://www.mountainproject.com/user/200180658/brayden-l'
upload_type = 'todo'
df_ulist = upload_df(upload_type, upload_link)

--- DATA CLEANSE AND STANDARDIZE ---

In [58]:
df_ulist = data_standardize(df_ulist)

--- GRADE HOMOGENIZATION AND ROUTE LENGTH CLEANUP ---

In [59]:
df_ulist = route_length_fixer(df_ulist)

Would you like to skip manual length assignment and assign a common value of 70ft for routes and 12ft for boulders to missing values?
Please select one of: y, n


In [ ]:
df_ulist, grade_homo_settings = grade_homo(df_ulist)

--- SCRAPE ---

In [ ]:
df_ulist = route_scrape(df_ulist)

--- ANALYZE ---

In [ ]:
df_ulist = extract_tick_details(df_ulist)

In [63]:
# Let's save the now scraped dataframe to a pickle file
df_ulist.to_pickle('../Data_Archive/df_todo_archive')

In [101]:
# To load the new pickle file
picklefile = open('../Data_Archive/df_todo_archive', 'rb')
df_ulist = pickle.load(picklefile)

In [ ]:
df_ulist_r = tick_analysis(df_ulist)

--- SPLIT OUTPUT INTO BOULDER AND ROPED SUBFRAME ---

In [103]:
df_ulist_r = df_ulist[df_ulist['Route Type'] != 'Boulder']
df_ulist_b = df_ulist[df_ulist['Route Type'] == 'Boulder']

In [104]:
min_ticks = 30

# Rarely led
df_low_lead = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['Lead Ratio'] < 0.4) & (df_ulist_r['Pitches'] == 1)].sort_values(by='Lead Ratio')

# Rarely toproped
df_high_lead = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['Lead Ratio'] > 0.9) & (df_ulist_r['Pitches'] == 1)].sort_values(by='Lead Ratio', ascending=False)

# Hard to OS
df_low_OS_r = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['OS Ratio'] < 0.35)].sort_values(by='OS Ratio')
df_low_OS_b = df_ulist_b[(df_ulist_b['Num Ticks'] >= min_ticks) & (df_ulist_b['OS Ratio'] < 0.35)].sort_values(by='OS Ratio')

# High OS
df_high_OS_r = df_ulist_r[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist_r['OS Ratio'] > 0.8)].sort_values(by='OS Ratio', ascending=False)
df_high_OS_b = df_ulist_b[(df_ulist_b['Num Ticks'] >= min_ticks) & (df_ulist_b['OS Ratio'] > 0.8)].sort_values(by='OS Ratio', ascending=False)

# Find route on list that is hardest and easiest to OS given a grade
OS_cutoff_num = 3
df_high_OS = df_ulist_r

In [119]:

grouplist = df_ulist['Rating'].unique()
outlist = []
for group in grouplist:
    outlist.extend(list(df_ulist[(df_ulist_r['Num Ticks'] >= min_ticks) & (df_ulist['Rating'] == group)].nsmallest(10, 'OS Ratio').index))
df_ulist.loc[outlist].sort_values('OS Ratio')

,Route,Original Rating,Rating,URL,Pitches,Location,Avg Stars,Your Stars,Route Type,Your Rating,Length,Rating Code,Route ID,Risk,Re mainpage,Re statpage,Re Mainpage,Re Statpage,Route Ticks,Num Ticks,Num Tickers,Lead Ratio,OS Ratio
897,Desert Gold,5.13a,5.13a,https://www.mountainproject.com/route/10593491...,2,Nevada > Southern Nevada > Red Rocks > 16-Blac...,4.0,-1,Trad,NaN,150.0,8600,105934919,PG13,,,<Response [200]>,<Response [200]>,Username ...,45.0,42.0,0.928571,0.000000
730,Galen's Crack,5.10c,5.10c,https://www.mountainproject.com/route/10767604...,1,California > Yosemite National Park > Tuolumne...,3.1,-1,Trad,NaN,45.0,3200,107676048,,,,<Response [200]>,<Response [200]>,Username ...,127.0,102.0,0.091837,0.000000
872,Make or Break Flake,5.10b,5.10b,https://www.mountainproject.com/route/10572552...,1,California > Joshua Tree National Park > Quail...,2.3,-1,Trad,NaN,60.0,2900,105725521,,,,<Response [200]>,<Response [200]>,Username ...,39.0,35.0,0.466667,0.000000
53,Hot Fun Sunday (Fell Down),5.11,5.11,https://www.mountainproject.com/route/10571880...,1,Utah > Southeast Utah > Indian Creek > Fin Wall,3.0,-1,Trad,NaN,70.0,5000,105718804,,,,<Response [200]>,<Response [200]>,Username ...,43.0,36.0,0.500000,0.000000
322,Crash and Burn,5.12d,5.12d,https://www.mountainproject.com/route/10662024...,1,California > Los Angeles Basin > Santa Monica ...,3.7,-1,Sport,NaN,50.0,7500,106620240,,,,<Response [200]>,<Response [200]>,Username ...,96.0,63.0,0.977778,0.016129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,[Redacted],5.10a/b,5.10a/b,https://www.mountainproject.com/route/10595364...,1,Nevada > Southern Nevada > Red Rocks > 01-Cali...,2.9,-1,Trad,NaN,80.0,2800,105953648,,,,<Response [200]>,<Response [200]>,Username ...,195.0,175.0,0.664062,0.945205
517,Couldn't be Schmooter,5.9-,5.9-,https://www.mountainproject.com/route/10593493...,1,Nevada > Southern Nevada > Red Rocks > 01-Cali...,3.2,-1,Trad,NaN,80.0,2300,105934930,,,,<Response [200]>,<Response [200]>,Username ...,371.0,333.0,0.704641,0.950355
741,West Ridge,5.4,5.4,https://www.mountainproject.com/route/10586939...,1,International > North America > Canada > Briti...,3.9,-1,Trad,NaN,1500.0,1400,105869393,,,,<Response [200]>,<Response [200]>,Username ...,611.0,541.0,0.786802,0.977273
711,Hexentric,5.7,5.7,https://www.mountainproject.com/route/10778399...,1,California > Sonora Pass Highway (108) > Upper...,1.4,-1,Trad,NaN,35.0,1800,107783999,,,,<Response [200]>,<Response [200]>,Username ...,33.0,32.0,0.730769,1.000000
